In [8]:
from google.colab import files
uploaded = files.upload()

Saving usage.csv to usage (1).csv
Saving transactions.csv to transactions (1).csv
Saving subscriptions.csv to subscriptions (1).csv
Saving customers.csv to customers (1).csv
Saving campaigns.csv to campaigns (2).csv


# Install DuckDB

**DuckDB** is a fast, open-source, embedded analytical database that runs within your application process (like Python or R) without requiring a separate server

In [9]:
!pip -q install duckdb

# Load CSVs + Register as SQL tables

In [10]:
import duckdb
import pandas as pd

con = duckdb.connect()

customers = pd.read_csv("customers.csv", parse_dates=["signup_date"])
subscriptions = pd.read_csv("subscriptions.csv", parse_dates=["signup_date"])
transactions = pd.read_csv("transactions.csv", parse_dates=["txn_date"])
usage = pd.read_csv("usage.csv", parse_dates=["event_date"])
campaigns = pd.read_csv("campaigns.csv", parse_dates=["exposure_date"])

con.register("customers", customers)
con.register("subscriptions", subscriptions)
con.register("transactions", transactions)
con.register("usage", usage)
con.register("campaigns", campaigns)

print("Tables registered:", con.execute("SHOW TABLES").fetchdf())

Tables registered:             name
0      campaigns
1      customers
2  subscriptions
3   transactions
4          usage


In [12]:
SNAPSHOT_DATE = "2024-04-15" # Choosing a Snapshot Date

# Building Features in SQL (RFM + Usage + Campaign History)

In [14]:
con.execute(f"""
CREATE OR REPLACE TABLE features AS
WITH base AS (
  SELECT
    c.customer_id,
    DATE '{SNAPSHOT_DATE}' AS snapshot_date,
    c.signup_date,
    c.country,
    c.plan_type,
    c.device,
    DATE_DIFF('day', c.signup_date, DATE '{SNAPSHOT_DATE}') AS tenure_days
  FROM customers c
),
tx AS (
  SELECT
    customer_id,
    MAX(txn_date) AS last_txn_date,
    COUNT(*) FILTER (WHERE txn_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 30 DAY AND DATE '{SNAPSHOT_DATE}') AS txn_cnt_30d,
    SUM(amount) FILTER (WHERE txn_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 30 DAY AND DATE '{SNAPSHOT_DATE}') AS spend_30d,
    SUM(amount) FILTER (WHERE txn_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 90 DAY AND DATE '{SNAPSHOT_DATE}') AS spend_90d
  FROM transactions
  WHERE txn_date <= DATE '{SNAPSHOT_DATE}'
  GROUP BY customer_id
),
usg AS (
  SELECT
    customer_id,
    SUM(minutes_used) FILTER (WHERE event_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 7 DAY AND DATE '{SNAPSHOT_DATE}') AS usage_minutes_7d,
    SUM(minutes_used) FILTER (WHERE event_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 30 DAY AND DATE '{SNAPSHOT_DATE}') AS usage_minutes_30d,
    COUNT(*) FILTER (WHERE event_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 30 DAY AND DATE '{SNAPSHOT_DATE}') AS active_days_30d,
    MAX(event_date) AS last_event_date
  FROM usage
  WHERE event_date <= DATE '{SNAPSHOT_DATE}'
  GROUP BY customer_id
),
camp_hist AS (
  SELECT
    customer_id,
    COUNT(*) FILTER (WHERE exposure_date BETWEEN DATE '{SNAPSHOT_DATE}' - INTERVAL 90 DAY AND DATE '{SNAPSHOT_DATE}') AS exposures_90d,
    MAX(exposure_date) AS last_exposure_date
  FROM campaigns
  WHERE exposure_date <= DATE '{SNAPSHOT_DATE}'
  GROUP BY customer_id
)
SELECT
  b.*,
  -- Recency features
  DATE_DIFF('day', COALESCE(tx.last_txn_date, b.signup_date), b.snapshot_date) AS recency_txn_days,
  DATE_DIFF('day', COALESCE(usg.last_event_date, b.signup_date), b.snapshot_date) AS recency_usage_days,

  -- Frequency / Monetary
  COALESCE(tx.txn_cnt_30d, 0) AS txn_cnt_30d,
  COALESCE(tx.spend_30d, 0.0) AS spend_30d,
  COALESCE(tx.spend_90d, 0.0) AS spend_90d,

  -- Usage
  COALESCE(usg.usage_minutes_7d, 0) AS usage_minutes_7d,
  COALESCE(usg.usage_minutes_30d, 0) AS usage_minutes_30d,
  COALESCE(usg.active_days_30d, 0) AS active_days_30d,

  -- Campaign history
  COALESCE(ch.exposures_90d, 0) AS exposures_90d,
  DATE_DIFF('day', COALESCE(ch.last_exposure_date, b.signup_date), b.snapshot_date) AS recency_exposure_days

FROM base b
LEFT JOIN tx ON b.customer_id = tx.customer_id
LEFT JOIN usg ON b.customer_id = usg.customer_id
LEFT JOIN camp_hist ch ON b.customer_id = ch.customer_id
""")
con.execute("SELECT * FROM features LIMIT 5").fetchdf()

,customer_id,snapshot_date,signup_date,country,plan_type,device,tenure_days,recency_txn_days,recency_usage_days,txn_cnt_30d,spend_30d,spend_90d,usage_minutes_7d,usage_minutes_30d,active_days_30d,exposures_90d,recency_exposure_days
0,11,2024-04-15,2023-11-27,Canada,Standard,iOS,140,34,7,0,0.0,55.0,112.0,786.0,12,1,29
1,15,2024-04-15,2023-06-01,UK,Premium,iOS,319,7,0,1,25.0,78.0,160.0,385.0,9,1,17
2,18,2024-04-15,2023-11-05,UK,Premium,iOS,162,15,4,2,26.0,51.0,90.0,224.0,4,1,5
3,22,2024-04-15,2023-07-11,Canada,Standard,Android,279,44,1,0,0.0,98.0,292.0,692.0,10,1,38
4,26,2024-04-15,2023-01-22,UK,Premium,iOS,449,28,2,2,30.0,117.0,136.0,319.0,5,1,9


# Creating Labels in SQL

**Label 1:** **Conversion (30 days after exposure)**

We’ll define conversion for each customer if they had any purchase within 30 days after their latest exposure before snapshot.

In [15]:
con.execute(f"""
CREATE OR REPLACE TABLE labels_conversion AS
WITH last_exp AS (
  SELECT
    customer_id,
    MAX(exposure_date) AS last_exposure_date
  FROM campaigns
  WHERE exposure_date <= DATE '{SNAPSHOT_DATE}'
  GROUP BY customer_id
),
conv AS (
  SELECT
    e.customer_id,
    CASE WHEN COUNT(t.*) > 0 THEN 1 ELSE 0 END AS converted_30d
  FROM last_exp e
  LEFT JOIN transactions t
    ON t.customer_id = e.customer_id
   AND t.txn_date > e.last_exposure_date
   AND t.txn_date <= e.last_exposure_date + INTERVAL 30 DAY
  GROUP BY e.customer_id
)
SELECT * FROM conv
""")

**Label 2: Churn (60 days after snapshot)**

Churn = no activity in usage for 60 days after snapshot.

In [16]:
con.execute(f"""
CREATE OR REPLACE TABLE labels_churn AS
WITH future_activity AS (
  SELECT
    customer_id,
    COUNT(*) AS events_next_60d
  FROM usage
  WHERE event_date > DATE '{SNAPSHOT_DATE}'
    AND event_date <= DATE '{SNAPSHOT_DATE}' + INTERVAL 60 DAY
  GROUP BY customer_id
)
SELECT
  f.customer_id,
  CASE WHEN COALESCE(a.events_next_60d, 0) = 0 THEN 1 ELSE 0 END AS churn_60d
FROM (SELECT customer_id FROM customers) f
LEFT JOIN future_activity a USING(customer_id)
""")

**Label 3: LTV (6 months & 12 months after snapshot)**

LTV = total transaction amount in next 6/12 months.

In [17]:
con.execute(f"""
CREATE OR REPLACE TABLE labels_ltv AS
SELECT
  c.customer_id,
  COALESCE(SUM(t.amount) FILTER (
    WHERE t.txn_date > DATE '{SNAPSHOT_DATE}'
      AND t.txn_date <= DATE '{SNAPSHOT_DATE}' + INTERVAL 180 DAY
  ), 0.0) AS ltv_6m,

  COALESCE(SUM(t.amount) FILTER (
    WHERE t.txn_date > DATE '{SNAPSHOT_DATE}'
      AND t.txn_date <= DATE '{SNAPSHOT_DATE}' + INTERVAL 365 DAY
  ), 0.0) AS ltv_12m
FROM customers c
LEFT JOIN transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id
""")

# Build Final Modeling Table (features + labels)

In [18]:
con.execute("""
CREATE OR REPLACE TABLE model_table AS
SELECT
  f.*,
  COALESCE(cv.converted_30d, 0) AS converted_30d,
  ch.churn_60d,
  l.ltv_6m,
  l.ltv_12m
FROM features f
LEFT JOIN labels_conversion cv USING(customer_id)
LEFT JOIN labels_churn ch USING(customer_id)
LEFT JOIN labels_ltv l USING(customer_id)
""")

model_df = con.execute("SELECT * FROM model_table").fetchdf()
model_df.head()

,customer_id,snapshot_date,signup_date,country,plan_type,device,tenure_days,recency_txn_days,recency_usage_days,txn_cnt_30d,...,spend_90d,usage_minutes_7d,usage_minutes_30d,active_days_30d,exposures_90d,recency_exposure_days,converted_30d,churn_60d,ltv_6m,ltv_12m
0,11,2024-04-15,2023-11-27,Canada,Standard,iOS,140,34,7,0,...,55.0,112.0,786.0,12,1,29,1,0,139.0,139.0
1,15,2024-04-15,2023-06-01,UK,Premium,iOS,319,7,0,1,...,78.0,160.0,385.0,9,1,17,1,0,76.0,76.0
2,18,2024-04-15,2023-11-05,UK,Premium,iOS,162,15,4,2,...,51.0,90.0,224.0,4,1,5,1,0,87.0,87.0
3,22,2024-04-15,2023-07-11,Canada,Standard,Android,279,44,1,0,...,98.0,292.0,692.0,10,1,38,1,0,48.0,48.0
4,26,2024-04-15,2023-01-22,UK,Premium,iOS,449,28,2,2,...,117.0,136.0,319.0,5,1,9,1,0,61.0,61.0


# Quick sanity checks

In [19]:
print("Rows:", len(model_df))
print("\nMissing values (top):")
print(model_df.isna().sum().sort_values(ascending=False).head(10))

print("\nLabel rates:")
print("converted_30d rate:", model_df["converted_30d"].mean())
print("churn_60d rate:", model_df["churn_60d"].mean())
print("avg ltv_6m:", model_df["ltv_6m"].mean())
print("avg ltv_12m:", model_df["ltv_12m"].mean())

Rows: 5000

Missing values (top):
customer_id           0
snapshot_date         0
signup_date           0
country               0
plan_type             0
device                0
tenure_days           0
recency_txn_days      0
recency_usage_days    0
txn_cnt_30d           0
dtype: int64

Label rates:
converted_30d rate: 0.4634
churn_60d rate: 0.0002
avg ltv_6m: 45.8228
avg ltv_12m: 45.8228


# Saving the file for Modeling

In [20]:
model_df.to_csv("model_table.csv", index=False)
print("Saved: model_table.csv ✅")

Saved: model_table.csv ✅
